In [1]:
import pandas as pd
import openml
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from algorithms.adaboost import AdaBoost


### Obter datasets do OpenML




In [2]:
def obter_dataset(id):


    dataset = openml.datasets.get_dataset(id, download_data=True, download_qualities=True, download_features_meta_data=True)

    X, y, _, attrs = dataset.get_data(dataset_format="array", target=dataset.default_target_attribute)

    df = pd.DataFrame(X, columns=attrs)
    df['target'] = y
    # converter:
    #   0 -> -1
    #   1 -> 1
    df['target'] = 2*y-1
    # erase rows with NaN values
    df = df.dropna(how='any', axis=0)

    csv_path = f'{id}.csv'
    df.to_csv(csv_path, index= False)

    return df

In [3]:
list_num_dataset_id = [1464, 1067, 1467, 1504,1494, 1068, 44, 1471, 1046, 1050, 1462, 151, 1485, 1461]
dataframes = {}
for dataset_id in list_num_dataset_id:
    df = obter_dataset(dataset_id)
    dataframes[dataset_id] = df

C:\Users\ricar\AppData\Local\Temp\ipykernel_6208\455471886.py:6: FutureWarning: Support for `dataset_format='array'` will be removed in 0.15,start using `dataset_format='dataframe' to ensure your code will continue to work. You can use the dataframe's `to_numpy` function to continue using numpy arrays.
  X, y, _, attrs = dataset.get_data(dataset_format="array", target=dataset.default_target_attribute)
C:\Users\ricar\AppData\Local\Temp\ipykernel_6208\455471886.py:6: FutureWarning: Support for `dataset_format='array'` will be removed in 0.15,start using `dataset_format='dataframe' to ensure your code will continue to work. You can use the dataframe's `to_numpy` function to continue using numpy arrays.
  X, y, _, attrs = dataset.get_data(dataset_format="array", target=dataset.default_target_attribute)
C:\Users\ricar\AppData\Local\Temp\ipykernel_6208\455471886.py:6: FutureWarning: Support for `dataset_format='array'` will be removed in 0.15,start using `dataset_format='dataframe' to ensure

### Análise dos datasets

O valor que você deve colocar no limite depende do que você considera um outlier. No código que você forneceu, você está usando a regra do escore z para detectar outliers. Especificamente, você está considerando qualquer valor que seja mais de 3 desvios padrão da média como um outlier.  

O valor de 3 é comumente usado na regra do escore z porque corresponde a um nível de confiança de cerca de 99,7% em uma distribuição normal. Isso significa que, em uma distribuição normal, esperamos que cerca de 99,7% dos valores estejam dentro de 3 desvios padrão da média.



Quando estamos a analisar os outliers fará sentido separar as classes (ver o boxplot para cada classe separadamente) ou é melhor analisar todas as classes juntas?
A análise de outliers deve ser feita para cada classe separadamente. Se você analisar todas as classes juntas,
 você pode acabar considerando valores que são normais para uma classe como outliers, simplesmente porque eles são incomuns em relação a outra classe.



In [5]:
# Análise dos outliers de cada dataset 

def showoutliers(ids, show_Boxplot = True):

    limite = 5

    df_1 = dataframes[id][dataframes[id]['target'] == 1]
    df_1 = df_1.drop(columns=['target'], axis=1)
    df_not_1 = dataframes[id][dataframes[id]['target'] == -1]
    df_not_1 = df_not_1.drop(columns=['target'], axis=1)

    outliers_1 = set()
    outliers_not_1 = set()

    for coluna in df_1.columns:

        data_plot_1 = df_1[coluna]
        data_plot_not_1 = df_not_1[coluna]

        if show_Boxplot:
            print(f'Outliers em {coluna}:')
            fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

            # Plot para a classe 1
            axs[0].boxplot(df_1[f'{coluna}'], vert=False)
            axs[0].set_title('Boxplot para df_1')

            # Plot para a classe -1
            axs[1].boxplot(df_not_1[f'{coluna}'], vert=False)
            axs[1].set_title('Boxplot para df_not_1')

            plt.show()

        z_scores_1 = np.abs((data_plot_1 - data_plot_1.mean()) / data_plot_1.std())
        outliers_1.update(data_plot_1[z_scores_1 > limite].index)
        #print(f"outliers 1: {outliers_1}")

        z_scores_not_1 = np.abs((data_plot_not_1 - data_plot_not_1.mean()) / data_plot_not_1.std())
        outliers_not_1.update(data_plot_not_1[z_scores_not_1 > limite].index)
        #print(f"outliers -1: {outliers_not_1}")

    n_outliers = len(outliers_1) + len(outliers_not_1)
    return n_outliers

In [6]:
for id in list_num_dataset_id:
    n_outliers = showoutliers(id, show_Boxplot=False)
    print(f"\nDataset {id}:")
    print(f'Número de outliers: {n_outliers}')
    print(f'Número de entradas: {len(dataframes[id])}')
    #show % of outliers in the dataset
    percentage_outliers = n_outliers/(len(dataframes[id]))
    print(f'Percentagem de outliers: {percentage_outliers*100:.2f}%')


Dataset 1464:
Número de outliers: 5
Número de entradas: 748
Percentagem de outliers: 0.67%

Dataset 1067:
Número de outliers: 88
Número de entradas: 2109
Percentagem de outliers: 4.17%

Dataset 1467:
Número de outliers: 0
Número de entradas: 540
Percentagem de outliers: 0.00%

Dataset 1504:
Número de outliers: 45
Número de entradas: 1941
Percentagem de outliers: 2.32%

Dataset 1494:
Número de outliers: 141
Número de entradas: 1055
Percentagem de outliers: 13.36%

Dataset 1068:
Número de outliers: 38
Número de entradas: 1109
Percentagem de outliers: 3.43%

Dataset 44:
Número de outliers: 1227
Número de entradas: 4601
Percentagem de outliers: 26.67%

Dataset 1471:
Número de outliers: 6
Número de entradas: 14980
Percentagem de outliers: 0.04%

Dataset 1046:
Número de outliers: 92
Número de entradas: 15545
Percentagem de outliers: 0.59%

Dataset 1050:
Número de outliers: 69
Número de entradas: 1563
Percentagem de outliers: 4.41%

Dataset 1462:
Número de outliers: 0
Número de entradas: 137

In [ ]:
def run_cv(X, y, algs, nfolds=10):
    results_dict = {}
    kf = KFold(n_splits=nfolds, shuffle=True, random_state=1111)
    for algo_name, algo in algs:
        sum_fold = 0
        number_of_outliers = 0
        for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            algo.fit(X_train, y_train)
            y_pred = algo.predict(X_test)
            sum_fold += accuracy_score(y_test, y_pred)
            if algo_name == "AdaBoost Check Outliers 3" or algo_name == "AdaBoost Check Outliers 5":
                number_of_outliers += algo.get_number_of_outliers()
        results_dict[algo_name] = sum_fold/nfolds
        if algo_name == "AdaBoost Check Outliers 3" or algo_name == "AdaBoost Check Outliers 5":
            results_dict["% of outliers"] = number_of_outliers/nfolds
    return results_dict

In [ ]:
models = [
    ("AdaBoost ficheiro", AdaBoost()),
    ("AdaBoost Check Outliers 3", AdaBoost(check_outliers=True, outlier_threshold=3)),
    ("AdaBoost Check Outliers 5", AdaBoost(check_outliers=True, outlier_threshold=5)),
    ("AdaBoost biblioteca", AdaBoostClassifier(algorithm='SAMME'))
]

results_list = []
for id in list_num_dataset_id:
    X = dataframes[id].drop(columns=['target'], axis=1)
    y = dataframes[id]['target']
    results = run_cv(X, y, models)
    results_list.append(results)



In [ ]:
# Convert list of dictionaries to DataFrame
df = pd.DataFrame(results_list)

# Calculate differences
df["diff original_3"] = abs(df["AdaBoost ficheiro"] - df["AdaBoost Check Outliers 3"])
df["diff original_5"] = abs(df["AdaBoost ficheiro"] - df["AdaBoost Check Outliers 5"])

df

In [4]:
from sklearn.model_selection import train_test_split

models = [
    ("AdaBoost ficheiro", AdaBoost()),
    ("AdaBoost Check Outliers 3", AdaBoost(check_outliers=True, outlier_threshold=3)),
    ("AdaBoost Check Outliers 5", AdaBoost(check_outliers=True, outlier_threshold=5)),
    ("AdaBoost biblioteca", AdaBoostClassifier(algorithm='SAMME'))
]

for id in list_num_dataset_id:
    X = dataframes[id].drop(columns=['target'], axis=1)
    y = dataframes[id]['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
    for algo_name, algo in models:
        number_of_outliers = 0
        algo.fit(X_train, y_train)
        y_pred = algo.predict(X_test)
        score = accuracy_score(y_test, y_pred)  # Changed variable name here
        if algo_name == "AdaBoost Check Outliers 3" or algo_name == "AdaBoost Check Outliers 5":
            number_of_outliers += algo.get_number_of_outliers()
        print(f"Dataset {id} - {algo_name}:")
        print(f"Accuracy: {score}")  # And here
        print("\n")
    print("\n")

Dataset 1464 - AdaBoost ficheiro:
Accuracy: 0.7377777777777778

Dataset 1464 - AdaBoost Check Outliers 3:
Accuracy: 0.7377777777777778

Dataset 1464 - AdaBoost Check Outliers 5:
Accuracy: 0.7377777777777778


Dataset 1464 - AdaBoost biblioteca:
Accuracy: 0.7866666666666666



Dataset 1067 - AdaBoost ficheiro:
Accuracy: 0.8515007898894155

Dataset 1067 - AdaBoost Check Outliers 3:
Accuracy: 0.8214849921011058

Dataset 1067 - AdaBoost Check Outliers 5:
Accuracy: 0.8451816745655608

Dataset 1067 - AdaBoost biblioteca:
Accuracy: 0.8515007898894155



Dataset 1467 - AdaBoost ficheiro:
Accuracy: 0.9135802469135802

Dataset 1467 - AdaBoost Check Outliers 3:
Accuracy: 0.9135802469135802

Dataset 1467 - AdaBoost Check Outliers 5:
Accuracy: 0.9074074074074074

Dataset 1467 - AdaBoost biblioteca:
Accuracy: 0.8950617283950617



Dataset 1504 - AdaBoost ficheiro:
Accuracy: 0.7650085763293311

Dataset 1504 - AdaBoost Check Outliers 3:
Accuracy: 0.7238421955403087

Dataset 1504 - AdaBoost Check Outli